In [37]:
import os
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [38]:
cd /content/drive/MyDrive/weather_bigdata_contest/code/

/content/drive/MyDrive/weather_bigdata_contest/code


In [72]:
import pandas as pd
from sklearn.multiclass import OneVsRestClassifier
from xgboost import XGBClassifier
from sklearn.preprocessing import MultiLabelBinarizer
from xgboost import XGBRegressor
from sklearn.multioutput import MultiOutputRegressor
from sklearn.metrics import mean_squared_error
import numpy as np

In [40]:
X_train = pd.read_csv('../data/X_train.csv', encoding = 'CP949')
X_test = pd.read_csv('../data/X_test.csv', encoding = 'CP949')
y_train = pd.read_csv('../data/y_train.csv', encoding = 'CP949')
y_test = pd.read_csv('../data/y_test.csv', encoding = 'CP949')

In [41]:
X_test.head()

,날짜,소분류_가스온수기,소분류_가열식 가습기,소분류_감/홍시,소분류_감귤/한라봉/오렌지,소분류_감말랭이,소분류_감자,소분류_건대추,소분류_건포도,소분류_견과류 밤,소분류_계란,소분류_고추/피망/파프리카,소분류_곡물차,소분류_공기정화 용품,소분류_공기청정기,소분류_과일류,소분류_과일채소 분말/분태,소분류_굴 생물,소분류_기능성 아이케어 화장품,소분류_기능성 화이트닝 화장품,소분류_기초 화장용 로션,소분류_기초 화장용 크림,소분류_기타 주스류,소분류_김치류,소분류_난방용 열풍기,소분류_남성 로션,소분류_남성 선케어,소분류_남성 세트,소분류_남성향수,소분류_냉풍기,소분류_네일 메이크업 용품,소분류_다이어트용 헬스보충식품,소분류_대게/킹크랩,소분류_더치커피,소분류_데오드란트,소분류_돈풍기,소분류_둥굴레차,소분류_딸기/복분자/블루베리,소분류_라디에이터,소분류_레몬/자몽,...,소분류_입욕제,소분류_자연식 가습기,소분류_장어,소분류_전기온수기,소분류_전기장판,소분류_전통차,소분류_절임배추/김치속,소분류_제습기,소분류_조개,소분류_중대형 에어컨,소분류_참외/메론/수박,소분류_천장형 에어컨,소분류_초음파식 가습기,소분류_카페 푸드,소분류_카페트매트,소분류_컨벡터,소분류_코코아/핫초코,소분류_키위/참다래,소분류_탁상/USB 선풍기,소분류_탄산수,소분류_탄산음료,소분류_토마토,소분류_풋크림,소분류_해초류,소분류_핸드크림,소분류_향수세트,소분류_허브차,소분류_헤어스타일링용 흑채,소분류_홍차,소분류_황토매트,소분류_회,소분류_휴대용 선풍기,소분류_히터,평균기온(°C),최저기온(°C),최고기온(°C),최소 상대습도(%),평균 상대습도(%),O3,PM10
0,2019-07-29,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,27.701529,25.230303,30.860849,63.957357,78.434966,0.013696,18.407551
1,2018-04-27,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,15.579250,10.169063,22.820054,31.936348,55.294139,0.036817,57.685314
2,2018-02-13,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,-0.068801,-5.822969,5.226839,32.841705,53.851700,0.020532,47.890027
3,2019-07-07,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,25.355820,22.052994,30.114279,37.545459,53.852888,0.042066,21.680626
4,2018-02-24,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,4.286319,0.593666,9.338614,22.932933,53.656280,0.025863,80.105869


In [7]:
y_train.head()

,20대 남성 판매량(개),20대 여성 판매량(개),30대 남성 판매량(개),30대 여성 판매량(개),40대 남성 판매량(개),40대 여성 판매량(개),50대 남성 판매량(개),50대 여성 판매량(개),60대 남성 판매량(개),60대 여성 판매량(개),합계,일별 물품 선호도 점수,20대 남성 선호도 점수,20대 여성 선호도 점수,30대 남성 선호도 점수,30대 여성 선호도 점수,40대 남성 선호도 점수,40대 여성 선호도 점수,50대 남성 선호도 점수,50대 여성 선호도 점수,60대 남성 선호도 점수,60대 여성 선호도 점수
0,0.0,1.0,2.0,0.0,1.0,4.0,1.0,4.0,0.0,1.0,14.0,0.000005,0.000000,0.041312,0.113754,0.000000,0.060858,0.112415,0.124614,0.262377,0.000000,0.284671
1,6.0,20.0,10.0,38.0,12.0,30.0,8.0,7.0,2.0,4.0,137.0,0.000037,0.092382,0.104007,0.071597,0.109999,0.091931,0.106132,0.125492,0.057800,0.097320,0.143339
2,0.0,0.0,1.0,7.0,1.0,7.0,0.0,0.0,0.0,0.0,16.0,0.000002,0.000000,0.000000,0.119632,0.338576,0.128006,0.413786,0.000000,0.000000,0.000000,0.000000
3,3.0,0.0,1.0,4.0,2.0,1.0,0.0,0.0,1.0,0.0,12.0,0.000005,0.348746,0.000000,0.054057,0.087421,0.115681,0.026710,0.000000,0.000000,0.367385,0.000000
4,7.0,88.0,30.0,142.0,43.0,188.0,25.0,76.0,3.0,13.0,615.0,0.000140,0.028234,0.119883,0.056268,0.107681,0.086296,0.174232,0.102733,0.164393,0.038242,0.122037


In [43]:
sale_category = ['y_sale_' + str(i) + '_' + j for i in range(20, 70, 10) for j in ['M', 'F']]
pref_category = ['y_pref_' + str(i) + '_' + j for i in range(20, 70, 10) for j in ['M', 'F']]

sale_category_k = [str(i) + '대 ' + j + ' 판매량(개)' for i in range(20, 70, 10) for j in ['남성', '여성']]
pref_category_k = [str(i) + '대 ' + j + ' 선호도 점수' for i in range(20, 70, 10) for j in ['남성', '여성']]

In [44]:
sale_category

['y_sale_20_M',
 'y_sale_20_F',
 'y_sale_30_M',
 'y_sale_30_F',
 'y_sale_40_M',
 'y_sale_40_F',
 'y_sale_50_M',
 'y_sale_50_F',
 'y_sale_60_M',
 'y_sale_60_F']

In [45]:
pref_category_k

['20대 남성 선호도 점수',
 '20대 여성 선호도 점수',
 '30대 남성 선호도 점수',
 '30대 여성 선호도 점수',
 '40대 남성 선호도 점수',
 '40대 여성 선호도 점수',
 '50대 남성 선호도 점수',
 '50대 여성 선호도 점수',
 '60대 남성 선호도 점수',
 '60대 여성 선호도 점수']

In [46]:
X_train.drop(['날짜'], axis = 1, inplace = True)
X_test.drop(['날짜'], axis = 1, inplace = True)

In [59]:
for i in range(len(pref_category_k)):
    globals()['xgb_rgr_pref_{}'.format(pref_category[i])] = XGBRegressor()
    globals()['xgb_rgr_pref_{}'.format(pref_category[i])].fit(X_train, y_train[pref_category_k[i]])
    globals()['pref_prediction_{}'.format(pref_category[i])] = globals()['xgb_rgr_pref_{}'.format(pref_category[i])].predict(X_test)

[03:14:29] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[03:14:46] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[03:15:02] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[03:15:19] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[03:15:36] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[03:15:53] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[03:16:10] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[03:16:27] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of re

In [64]:
pred_prediction = []
for i in category:
    pred_prediction.append(globals()['pref_prediction_{}'.format(i)])

In [65]:
np.array(pred_prediction)

array([[0.07589057, 0.07589057, 0.0796124 , ..., 0.07589057, 0.07589057,
        0.07589057],
       [0.07416415, 0.07416415, 0.07416415, ..., 0.3019315 , 0.07416415,
        0.07416415],
       [0.10984874, 0.12235281, 0.10984874, ..., 0.08491072, 0.10984874,
        0.10984874],
       ...,
       [0.08752385, 0.08752385, 0.08752385, ..., 0.08752385, 0.08752385,
        0.09920296],
       [0.0905427 , 0.09219471, 0.09219471, ..., 0.06044972, 0.09219471,
        0.09219471],
       [0.08392182, 0.08115855, 0.07756421, ..., 0.07448396, 0.07756421,
        0.09405223]], dtype=float32)

In [66]:
y_test[pref_category_k]

,20대 남성 선호도 점수,20대 여성 선호도 점수,30대 남성 선호도 점수,30대 여성 선호도 점수,40대 남성 선호도 점수,40대 여성 선호도 점수,50대 남성 선호도 점수,50대 여성 선호도 점수,60대 남성 선호도 점수,60대 여성 선호도 점수
0,0.000000,0.090174,0.124150,0.125486,0.182655,0.130356,0.204003,0.143177,0.000000,0.000000
1,0.000000,0.120017,0.132190,0.120251,0.070722,0.097976,0.217215,0.076225,0.000000,0.165404
2,0.107017,0.102411,0.091234,0.107305,0.133118,0.061473,0.127201,0.047826,0.056368,0.166046
3,0.000000,0.049952,0.034387,0.013903,0.091984,0.067964,0.188347,0.178456,0.116851,0.258158
4,0.052070,0.075758,0.057739,0.088858,0.069752,0.123322,0.073453,0.173989,0.126584,0.158475
...,...,...,...,...,...,...,...,...,...,...
24111,0.100995,0.034111,0.000000,0.018988,0.150752,0.116027,0.205788,0.054161,0.319178,0.000000
24112,0.024901,0.021026,0.032567,0.063786,0.084405,0.124442,0.126846,0.216166,0.088533,0.217328
24113,0.138452,0.277456,0.055797,0.119737,0.036740,0.116644,0.065826,0.074249,0.029170,0.085928
24114,0.000000,0.028927,0.079653,0.064408,0.085228,0.118073,0.087257,0.137791,0.000000,0.398663


In [57]:
np.array(pred_prediction).T

array([[0.07589057, 0.07416415, 0.10984874, ..., 0.08752385, 0.0905427 ,
        0.08392182],
       [0.07589057, 0.07416415, 0.12235281, ..., 0.08752385, 0.09219471,
        0.08115855],
       [0.0796124 , 0.07416415, 0.10984874, ..., 0.08752385, 0.09219471,
        0.07756421],
       ...,
       [0.07589057, 0.3019315 , 0.08491072, ..., 0.08752385, 0.06044972,
        0.07448396],
       [0.07589057, 0.07416415, 0.10984874, ..., 0.08752385, 0.09219471,
        0.07756421],
       [0.07589057, 0.07416415, 0.10984874, ..., 0.09920296, 0.09219471,
        0.09405223]], dtype=float32)

In [73]:
print('RMSE of preference prediction: {}'.format(str(np.sqrt(mean_squared_error(y_test[pref_category_k], np.array(pred_prediction).T)))))

RMSE of preference prediction: 0.11446944134053352


In [75]:
xgb_sale_prediction = XGBRegressor()
xgb_sale_prediction.fit(X_train, y_train['합계'])
sale_prediction = xgb_sale_prediction.predict(X_test)
print('RMSE of sale prediction: {}'.format(str(np.sqrt(mean_squared_error(y_test['합계'], sale_prediction)))))


[03:24:36] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
RMSE of sale prediction: 162.87807554004604
